

QUESTIONS:
What is our 30-day all cause readmission rate across hospitals our members are admitted to?
What percent of patients have a PCP visit within a week of discharge?
Do either vary by individual or group PCP's?

BONUS:
Which individual or PCP groups should we make an extra effort with to build a relationship? Which have the highest volume?
What percent of our patients do not have a PCP assigned or haven't seen their PCP recently?
Are there certain patients we should focus on (e.g. frequent fliers)?
From this, what could a version 1 of an actionable dashboard or intervention list look like to support the Transition Care Managers?

NOTES:

discharge date + 30 days: readmission true or false
count trues

UCSF = DS1|NPI|1992818256

In [1]:
import pandas as pd
import datetime as dt
import requests

In [2]:
members = pd.read_csv('../Data Files/NSS_GreatCare_Members.csv', parse_dates=[61,62], dtype={'Empirically Attributed PCP Individual Provider NPI':'object', 'Empirically Attributed PCP Group Provider NPI':'object'})
admissions = pd.read_csv('../Data Files/NSS_Hospital_Inpatient_Admissions.csv', parse_dates=[8,9])
pcp_visits = pd.read_csv('../Data Files/NSS_PrimaryCare_Office_Visits.csv', parse_dates=[12], dtype={'Service Provider NPI':'object', 'Provider Group NPI':'object'})

E:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (3,5,17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [5]:
admissions_dedup = admissions.sort_values(by=['Patient ID','Encounter Start Date', 'Encounter End Date']).drop_duplicates(subset=['Patient ID', 'Encounter Start Date'], keep = 'last')
admissions_dedup = admissions_dedup.sort_values(by=['Patient ID','Encounter End Date', 'Encounter Start Date']).drop_duplicates(subset=['Patient ID', 'Encounter End Date'], keep = 'first')
admissions.shape, admissions_dedup.shape

((6213, 23), (6149, 23))

In [5]:
pcp_visits[pcp_visits['Provider Group Provider ID'] == 'DS1|NPI|1992818256']

,Encounter ID,Patient ID,Service Provider ID,Attending Provider ID,Provider Group Provider ID,Facility Provider ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Attending Provider NPI,Facility Provider NPI,Service Provider NPI,Provider Group NPI,Encounter Date,Is Qualifying Preventive Primary Care Service,Is Qualifying Primary Care Service Provider,Is Qualifying Primary Care Attending,Is Qualifying Primary Care Provider Group,Qualifying Preventive Primary Care HCPCS Code


,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,CMS Admit Source Code,CMS Patient Discharge Status Code,CMS Place of Service Code,Principal Discharge Diagnosis ICD-9-CM Code,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,2,1,NaN,NaN,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.0,0.0
49,HIA|DS1|10494|DS1|NPI|1992818256|1,DS1|10494,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1861598690,1.861599e+09,1992818256,2016-11-27,2016-11-29,...,1,1,NaN,NaN,NaN,R23.0,Cyanosis,0,0.0,0.0
71,HIA|DS1|10707|DS1|NPI|1992818256|1,DS1|10707,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1194047126,1.194047e+09,1992818256,2010-04-19,2010-05-08,...,5,1,NaN,NaN,NaN,Q25.0,Patent ductus arteriosus,0,0.0,0.0
72,HIA|DS1|10707|DS1|NPI|1992818256|2,DS1|10707,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1932230810,1.932231e+09,1992818256,2010-07-25,2010-07-27,...,2,1,NaN,NaN,NaN,Q87.2,Congenital malformation syndromes predominantl...,0,0.0,0.0
73,HIA|DS1|10707|DS1|NPI|1992818256|3,DS1|10707,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1740437359,1.740437e+09,1992818256,2010-08-13,2010-08-16,...,2,1,NaN,NaN,NaN,Q60.2,"Renal agenesis, unspecified",0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5881,HIA|DS1|6775|DS1|NPI|1992818256|1,DS1|6775,DS1|1,DS1|5047X,DS1|NPI|1992818256,DS1|NPI|1063720431,1.063720e+09,1992818256,2015-04-27,2015-05-01,...,1,6,NaN,NaN,NaN,E87.1,Hypo-osmolality and hyponatremia,0,0.0,0.0
5939,HIA|DS1|7364|DS1|NPI|1992818256|1,DS1|7364,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1326347113,1.326347e+09,1992818256,2010-04-18,2010-04-21,...,2,1,NaN,NaN,NaN,Z3A.39,39 weeks gestation of pregnancy,0,0.0,0.0
6087,HIA|DS1|8886|DS1|NPI|1992818256|1,DS1|8886,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1720043417,1.720043e+09,1992818256,2012-03-14,2012-03-16,...,2,1,NaN,NaN,NaN,Z3A.39,39 weeks gestation of pregnancy,0,0.0,0.0
6115,HIA|DS1|9204|DS1|NPI|1992818256|1,DS1|9204,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1548325673,1.548326e+09,1992818256,2013-05-29,2013-05-30,...,2,1,NaN,NaN,NaN,Z3A.31,31 weeks gestation of pregnancy,0,0.0,0.0


In [6]:
def categorize_events(patient_admissions):
    patient_index_events = 0
    patient_readmission_events = 0
    last_index_event_discharge_date = None
    last_index_event_encounter_id = None
    readmission = False
    for ind, row in patient_admissions.iterrows():
        ##set first event as an intiial event or a non-qualifying readmission as initial event
        if patient_index_events == 0 or row['Encounter Start Date'] > last_index_event_discharge_date + dt.timedelta(days=30):
            patient_index_events += 1
            last_index_event_discharge_date = row['Encounter End Date']
            last_index_event_encounter_id = row['Encounter ID']
            readmission = False
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'index without readmission'
        ##check if qualifying readmission and change readmit status of most recently added initial event to true
        elif readmission == False:
            patient_readmission_events += 1
            readmission = True
            admissions.loc[admissions['Encounter ID'] == last_index_event_encounter_id, 'type'] = 'index w readmission'
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'initial readmission'
        ##check for duplicate readmission
        else:
            admissions.loc[admissions['Encounter ID'] == row['Encounter ID'], 'type'] = 'extra readmission'
            
    return patient_index_events, patient_readmission_events
    

In [7]:
total_index_events = 0
total_index_events_leading_to_readmission = 0

for patient in admissions_dedup['Patient ID'].unique():
    patient_admissions = admissions_dedup[admissions['Patient ID']==patient]
    times_admitted = patient_admissions.shape[0]
    if times_admitted > 1:
        patient_admissions.sort_values(by = 'Encounter End Date', inplace=True, ignore_index=True)
        ind, re = categorize_events(patient_admissions)
        total_index_events += ind
        total_index_events_leading_to_readmission += re
    else:
        encounter_id = patient_admissions['Encounter ID'].values[0]
        admissions.loc[admissions['Encounter ID'] == encounter_id, 'type'] = 'index without readmission'
        total_index_events += 1
print("Total Index Events:",total_index_events) 
print("Index Events Leading to Readmission:", total_index_events_leading_to_readmission)
print(str.format("Readmission Rate: {}%",round(total_index_events_leading_to_readmission/total_index_events*100,2)))

<ipython-input-7-45bfff96a76a>:5: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  patient_admissions = admissions_dedup[admissions['Patient ID']==patient]
<ipython-input-7-45bfff96a76a>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  patient_admissions.sort_values(by = 'Encounter End Date', inplace=True, ignore_index=True)


Total Index Events: 5536
Index Events Leading to Readmission: 527
Readmission Rate: 9.52%


In [8]:
admissions['type'].value_counts()

index without readmission    5009
initial readmission           527
index w readmission           527
extra readmission              86
Name: type, dtype: int64

In [9]:
admissions[['type']] = admissions[['type']].fillna(value='duplicate encounter')

In [10]:
admissions['type'].value_counts()

index without readmission    5009
initial readmission           527
index w readmission           527
extra readmission              86
duplicate encounter            64
Name: type, dtype: int64

In [11]:
total_hospital_visits = admissions_dedup.shape[0]
total_pcp_visits_in_7_days = 0

for ind, row in admissions_dedup.iterrows():
    patient = row['Patient ID']
    discharge_date = row['Encounter End Date']
    
    for i,r in pcp_visits[pcp_visits['Patient ID']==patient].iterrows():
        if r['Encounter Date'] <= discharge_date + dt.timedelta(days=7) and r['Encounter Date'] >= discharge_date:
            total_pcp_visits_in_7_days += 1
            admissions.loc[admissions['Encounter ID']==row['Encounter ID'], '7 day followup'] = True
            break
            
print('Total Hospital Visits: ', total_hospital_visits)
print('Total PCP Visits in 7 Days: ', total_pcp_visits_in_7_days)
print(str.format("PCP Visit Rate: {}%",round(total_pcp_visits_in_7_days/total_hospital_visits*100,2)))


Total Hospital Visits:  6149
Total PCP Visits in 7 Days:  1860
PCP Visit Rate: 30.25%


In [12]:
for ind, row in admissions.iterrows():
    patient = row['Patient ID']
    discharge = row['Encounter End Date']
    encounter = row['Encounter ID']
    mask = (pcp_visits['Patient ID'] == patient) & (pcp_visits['Encounter Date']>= discharge)
    patient_pcp = pcp_visits[mask].sort_values('Encounter Date')
    if len(patient_pcp['Encounter Date']) == 0:
        continue
    else:
        first_pcp_visit = patient_pcp.head(1)
        indiv_npi = first_pcp_visit['Service Provider NPI'].values[0]
        group_npi = first_pcp_visit['Provider Group NPI'].values[0]
        if indiv_npi == group_npi:
            admissions.loc[admissions['Encounter ID']==encounter, 'individual_npi'] = indiv_npi
        else:
            admissions.loc[admissions['Encounter ID']==encounter, 'individual_npi'] = indiv_npi
            admissions.loc[admissions['Encounter ID']==encounter, 'group_npi'] = group_npi

In [13]:
admissions[['7 day followup']] = admissions[['7 day followup']].fillna(value=False)
admissions.head()

,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,Encounter End Date,...,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,type,7 day followup,individual_npi,group_npi
0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,2013-01-14,...,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,index without readmission,False,1619980406,1043427248
1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,2014-01-12,...,Compression of brain,NaN,NaN,0,46465.40,250.0,index without readmission,True,1619199791,1043427248
2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,2012-06-21,...,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,index without readmission,True,1659394021,1043427248
3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,2012-12-09,...,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,index without readmission,True,1952538340,1043427248
4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,2015-03-14,...,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0,index without readmission,False,1316361942,1043427248


In [14]:
print(admissions['type'].value_counts())
admissions['individual_npi'].isna().sum(), admissions['group_npi'].isna().sum()

index without readmission    5009
initial readmission           527
index w readmission           527
extra readmission              86
duplicate encounter            64
Name: type, dtype: int64


(1288, 1380)

In [15]:
individual_mask = ~admissions['individual_npi'].isna()
index_events = admissions[individual_mask]['type'].value_counts()[0] + admissions[individual_mask]['type'].value_counts()[1]
readmissions = admissions[individual_mask]['type'].value_counts()[1]

non_dup = ~(admissions['type']=='duplicate encounter') & individual_mask
admission_events = admissions[non_dup].shape[0]
followup_7_days = admissions[(non_dup) & (admissions['7 day followup']==True)].shape[0]

print(f'Readmission rate for patients who see an individual PCP after discharge: {round(readmissions/index_events*100,2)}%')
print(f'7 day followup rate for patients who see an individual PCP after discharge: {round(followup_7_days/admission_events*100,2)}%')

Readmission rate for patients who see an individual PCP after discharge: 9.67%
7 day followup rate for patients who see an individual PCP after discharge: 38.01%


In [16]:
group_mask = ~admissions['group_npi'].isna()
index_events_g = admissions[group_mask]['type'].value_counts()[0] + admissions[group_mask]['type'].value_counts()[1]
readmissions_g = admissions[group_mask]['type'].value_counts()[1]

non_dup_g = ~(admissions['type']=='duplicate encounter') & group_mask
admission_events_g = admissions[non_dup_g].shape[0]
followup_7_days_g = admissions[(non_dup_g) & (admissions['7 day followup']==True)].shape[0]

print(f'Readmission rate for patients who see a group PCP after discharge: {round(readmissions_g/index_events_g*100,2)}%')
print(f'7 day followup rate for patients who see a group PCP after discharge: {round(followup_7_days_g/admission_events_g*100,2)}%')

Readmission rate for patients who see a group PCP after discharge: 9.63%
7 day followup rate for patients who see a group PCP after discharge: 38.21%


In [17]:
admissions['group_npi'].value_counts()

1043427248    2209
1184722779    1164
1164512851     481
1407058787     247
1316025091     173
              ... 
1699986950       1
1932283215       1
1922013804       1
1154354504       1
1437292927       1
Name: group_npi, Length: 116, dtype: int64

In [18]:
admissions['individual_npi'].value_counts()

1104884337    66
1619980406    52
1972704856    50
1467643700    50
1437180965    49
              ..
1184887408     1
1184609414     1
1558377481     1
1609882638     1
1265593701     1
Name: individual_npi, Length: 979, dtype: int64

In [19]:
admissions['Patient ID'].value_counts()

DS1|3812     17
DS1|26751    15
DS1|53241    13
DS1|10451    12
DS1|12422    11
             ..
DS1|59507     1
DS1|58697     1
DS1|6454      1
DS1|23466     1
DS1|54912     1
Name: Patient ID, Length: 4701, dtype: int64

In [20]:
members['Empirically Attributed PCP Individual Provider NPI'].isna().sum()

29680

In [21]:
members['Empirically Attributed PCP Group Provider NPI'].isna().sum()

29631

In [22]:
mask = members['Empirically Attributed PCP Individual Provider NPI'].isna() & members['Empirically Attributed PCP Group Provider NPI'].isna()
print(f'Members who do not have an assigned PCP (group or individual): {round(members[mask].shape[0]/members.shape[0]*100,2)}%')

Members who do not have an assigned PCP (group or individual): 47.96%


In [23]:
for member in members['Patient ID'].unique():
    member_pcp_visits = pcp_visits[pcp_visits['Patient ID']==member]
    most_recent_visit = member_pcp_visits['Encounter Date'].max()
    members.loc[members['Patient ID']==member, 'most recent visit'] = most_recent_visit

In [24]:
members.to_csv('../Data Files/leptons_members.csv')
admissions.to_csv('../Data Files/leptons_admissions.csv')
pcp_visits.to_csv('../Data Files/leptons_pcp.csv')

In [32]:
import pandas as pd

admis = pd.read_csv('../Data Files/leptons_admissions.csv', dtype={'individual_npi':'object', 'group_npi':'object'})
admis

,Unnamed: 0,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,...,Principal Discharge Diagnosis ICD-9-CM Description,Principal Discharge Diagnosis ICD-10-CM Code,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,type,7 day followup,individual_npi,group_npi
0,0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,...,NaN,O70.0,First degree perineal laceration during delivery,0,6620.72,250.0,index without readmission,False,1619980406,1043427248
1,1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,...,Compression of brain,NaN,NaN,0,46465.40,250.0,index without readmission,True,1619199791,1043427248
2,2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,...,NaN,K91.71,Accidental puncture and laceration of a digest...,0,43879.13,250.0,index without readmission,True,1659394021,1043427248
3,3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,...,NaN,O70.1,Second degree perineal laceration during delivery,0,14800.00,250.0,index without readmission,True,1952538340,1043427248
4,4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,...,NaN,F32.9,"Major depressive disorder, single episode, uns...",0,0.00,0.0,index without readmission,False,1316361942,1043427248
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,...,NaN,O44.13,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,index without readmission,True,1740277995,1477666352
6209,6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,...,"Second-degree perineal laceration, delivered, ...",NaN,NaN,0,14800.00,250.0,index without readmission,True,1992932388,1043427248
6210,6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,...,NaN,O76,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,index w readmission,False,1487734547,1407058787
6211,6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,...,NaN,O42.92,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,initial readmission,False,1487734547,1407058787


In [30]:
admis_pres = pd.read_csv('../Data Files/admis.csv')

,Unnamed: 0,Encounter ID,Patient ID,Encounter Primary Payor ID,Encounter Primary Plan ID,Facility Provider ID,Attending Provider ID,Attending Provider NPI,Facility Provider NPI,Encounter Start Date,...,Principal Discharge Diagnosis ICD-10-CM Description,Primary Plan Allowed Amount,Primary Plan Paid Amount,Patient Paid Amount,Attending Provider NPI Type,Facility Provider NPI Type,index_admis,pcp_visit,NPI_type,readmission
0,0,HIA|DS1|10033|DS1|NPI|1306890389|1,DS1|10033,DS1|1,DS1|5047W,DS1|NPI|1306890389,DS1|NPI|1417170523,1.417171e+09,1306890389,2013-01-12,...,First degree perineal laceration during delivery,0,6620.72,250.0,NPI-1,NPI-2,True,False,NPI-2,NaN
1,1,HIA|DS1|10043|DS1|NPI|1902803315|1,DS1|10043,DS1|1,DS1|5047Y,DS1|NPI|1902803315,DS1|NPI|1669599197,1.669599e+09,1902803315,2014-01-05,...,NaN,0,46465.40,250.0,NPI-1,NPI-2,True,True,NPI-2,NaN
2,2,HIA|DS1|10073|DS1|NPI|1902803315|1,DS1|10073,DS1|1,DS1|5047X,DS1|NPI|1902803315,DS1|NPI|1407880511,1.407881e+09,1902803315,2012-06-13,...,Accidental puncture and laceration of a digest...,0,43879.13,250.0,NPI-1,NPI-2,True,True,NPI-2,NaN
3,3,HIA|DS1|10101|DS1|NPI|1427055839|1,DS1|10101,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1407008907,1.407009e+09,1427055839,2012-12-06,...,Second degree perineal laceration during delivery,0,14800.00,250.0,NPI-1,NPI-2,True,True,NPI-2,NaN
4,4,HIA|DS1|10118|DS1|NPI|1992818256|1,DS1|10118,DS1|1,DS1|5047W,DS1|NPI|1992818256,DS1|NPI|1518045608,1.518046e+09,1992818256,2015-03-11,...,"Major depressive disorder, single episode, uns...",0,0.00,0.0,NPI-1,NPI-2,True,False,NPI-2,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6208,6208,HIA|DS1|9979|DS1|NPI|1689772592|1,DS1|9979,DS1|1,DS1|5047X,DS1|NPI|1689772592,DS1|NPI|1720043417,1.720043e+09,1689772592,2015-02-06,...,"Complete placenta previa with hemorrhage, thir...",0,23598.00,250.0,NPI-1,NPI-2,True,True,NPI-2,NaN
6209,6209,HIA|DS1|9981|DS1|NPI|1427055839|1,DS1|9981,DS1|1,DS1|5047W,DS1|NPI|1427055839,DS1|NPI|1215968474,1.215968e+09,1427055839,2014-08-07,...,NaN,0,14800.00,250.0,NPI-1,NPI-2,True,True,NPI-2,NaN
6210,6210,HIA|DS1|9994|DS1|NPI|1689608150|1,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1861831836,1.861832e+09,1689608150,2015-11-21,...,Abnormality in fetal heart rate and rhythm com...,0,17919.75,250.0,NPI-1,NPI-2,True,False,NPI-2,True
6211,6211,HIA|DS1|9994|DS1|NPI|1689608150|2,DS1|9994,DS1|1,DS1|5047W,DS1|NPI|1689608150,DS1|NPI|1750544516,1.750545e+09,1689608150,2015-11-24,...,"Full-term premature rupture of membranes, unsp...",0,19919.42,250.0,NPI-1,NPI-2,False,False,NPI-2,NaN


In [33]:
npi = pd.read_csv('../Data Files/npi.csv')

In [37]:
for i,r in admis.iterrows():
    i_npi = r['individual_npi']
    g_npi = r['group_npi']
    
    if len(npi.loc[npi['npi']==i_npi,'enumeration'])!=0:
        i_enum = npi.loc[npi['npi']==i_npi,'enumeration'].values[0]
        admis.loc[admis['Encounter ID'] == r['Encounter ID'], 'i_type'] = i_enum
        
    if len(npi.loc[npi['npi']==g_npi,'enumeration'])!=0:
        g_enum = npi.loc[npi['npi']==g_npi,'enumeration'].values[0]
        admis.loc[admis['Encounter ID'] == r['Encounter ID'], 'g_type'] = g_enum



In [39]:
admis['i_type'].value_counts()

NPI-1    4900
NPI-0      16
NPI-2       9
Name: i_type, dtype: int64

In [40]:
admis['g_type'].value_counts()

NPI-2    4816
NPI-1      12
NPI-0       5
Name: g_type, dtype: int64

In [44]:
import requests

endpoint = 'https://npiregistry.cms.hhs.gov/api/?version=2.0'
params = {'number': 1619980406}
res = requests.get(endpoint, params)

In [46]:
res.json()

{'result_count': 1,
 'results': [{'enumeration_type': 'NPI-1',
   'number': 1619980406,
   'last_updated_epoch': 1578053963,
   'created_epoch': 1155513600,
   'basic': {'first_name': 'BLAKE',
    'last_name': 'ALBAN',
    'middle_name': 'M',
    'credential': 'MD',
    'sole_proprietor': 'NO',
    'gender': 'M',
    'enumeration_date': '2006-08-14',
    'last_updated': '2020-01-03',
    'status': 'A',
    'name': 'ALBAN BLAKE',
    'certification_date': '2020-01-03'},
   'other_names': [],
   'addresses': [{'country_code': 'US',
     'country_name': 'United States',
     'address_purpose': 'LOCATION',
     'address_type': 'DOM',
     'address_1': '11318 NATIONAL BLVD',
     'address_2': '',
     'city': 'LOS ANGELES',
     'state': 'CA',
     'postal_code': '900643727',
     'telephone_number': '310-231-9150',
     'fax_number': '310-268-9319'},
    {'country_code': 'US',
     'country_name': 'United States',
     'address_purpose': 'MAILING',
     'address_type': 'DOM',
     'address

In [26]:
new_admis = pd.read_csv('../Data Files/leptons_admissions.csv', dtype={'Attending Provider NPI':'object'}, parse_dates=[9,10])
new_admis.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6213 entries, 0 to 6212
Data columns (total 28 columns):
 #   Column                                               Non-Null Count  Dtype         
---  ------                                               --------------  -----         
 0   Unnamed: 0                                           6213 non-null   int64         
 1   Encounter ID                                         6213 non-null   object        
 2   Patient ID                                           6213 non-null   object        
 3   Encounter Primary Payor ID                           6213 non-null   object        
 4   Encounter Primary Plan ID                            6213 non-null   object        
 5   Facility Provider ID                                 6213 non-null   object        
 6   Attending Provider ID                                5884 non-null   object        
 7   Attending Provider NPI                               5884 non-null   object        
 8 

In [30]:
npi = pd.read_csv('../Data Files/npi_and_addresses_cleaned.csv', dtype={'npi_number':'object'})
npi.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214 entries, 0 to 213
Data columns (total 21 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Facility NPI       214 non-null    object 
 1   total count        214 non-null    int64  
 2   readmit count      77 non-null     float64
 3   percent_readmits   77 non-null     float64
 4   pcp visit count    89 non-null     float64
 5   percent_pcps       89 non-null     float64
 6   facility provider  214 non-null    object 
 7   concat address     214 non-null    object 
 8   Unnamed: 0_y       206 non-null    float64
 9   npi_type           206 non-null    object 
 10  organization       206 non-null    object 
 11  name               206 non-null    object 
 12  taxonomy_1         206 non-null    object 
 13  taxonomy_2         206 non-null    object 
 14  taxonomy_3         206 non-null    object 
 15  taxonomy_4         206 non-null    object 
 16  taxonomy raw       206 non

In [55]:
for ind,row in new_admis.iterrows():
    attending = row['Attending Provider NPI'] 
    facility = row['Facility Provider NPI'] 
    if facility in npi['Facility NPI'].values:
        new_admis.iloc[ind,28] = npi.loc[npi['Facility NPI']==facility,['facility provider']]

In [57]:
new_admis.to_csv('new_admis_for_dash.csv')

In [63]:
test = pd.read_csv('../Data Files/admis.csv',dtype={'Attending Provider NPI':'object'}, parse_dates=[9,10])

In [67]:
new_test = test.merge(new_admis, how='left',on='Encounter ID')

In [68]:
new_test.to_csv('redo_new_admis_for_dash.csv')